In [1]:
import numpy as np
import tinygrad 
from extra.datasets import fetch_cifar
import time
import cv2
start =time.monotonic()
X_train, Y_train = fetch_cifar(train=True)

In [2]:
from tinygrad.nn import Linear
from tinygrad.nn import optim
from tinygrad.tensor import Tensor

class TinyCIFAR:
    def __init__ (self):
        self.l1 = Linear(3072,1024, bias=True)
        self.l2 = Linear(1024,1024,bias=True)
        self.l3 = Linear(1024,10, bias=True)

    def __call__ (self, x):
        x = self.l1(x)
        x = x.leakyrelu()
        x = self.l2(x)
        x = x.leakyrelu()
        x = self.l3(x)
        return x.log_softmax()
    
net = TinyCIFAR()

In [4]:
X_trains = X_train.reshape(5000, -1)
print(X_trains.shape)
Tensor.training= True

(5000, 30720)


In [ ]:
from extra.training import sparse_categorical_crossentropy
opt = optim.Adam([net.l1.weight, net.l2.weight, net.l3.weight])


BS=32

#run the modelieren

for step in range(1):
    samp=np.random.randint(0, 5000, 1)
    batch = Tensor(X_train[0], requires_grad=True)
    print(samp.shape)
    labels = Y_train[1]
    out = net(batch)
    loss = sparse_categorical_crossentropy(out,labels)
    opt.zero_grad() #NOTE:I'm unsure why it's giving me t.grad is not None (it was array shaping)
    loss.backward()
    opt.step()
    #accuracy dealings
    pred = np.argmax(out.numpy(), axis=-1)
    acc = pred == labels.mean()
    # if acc >= 1:
    #     break
    if step % 100 == 0:
        print(f'step{step} | loss: {loss.numpy()} | Accuracy: {acc}')



(1,)


AssertionError: can't expand (1024, 3072) into (1024, 30720)